In [28]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType, AgentExecutor, create_openai_functions_agent
import warnings
warnings.filterwarnings(action='ignore')
load_dotenv()

True

In [26]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
    temperature=0
) 

In [4]:
template = "What is the leadership url of {company}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
chain.invoke({"company": "FactSet"})

AIMessage(content='The leadership URL of FactSet is https://www.factset.com/about/leadership.')

In [5]:
chain.invoke({"company": "Nvidia"})

AIMessage(content='The leadership URL for Nvidia is: \nhttps://www.nvidia.com/en-us/about-nvidia/leadership/')

In [6]:
chain.invoke({"company": "Microsoft"})

AIMessage(content='The leadership URL of Microsoft is https://news.microsoft.com/executives/.')

In [45]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.google_search import GoogleSearchResults

In [46]:
search = GoogleSearchResults()

ValidationError: 1 validation error for GoogleSearchResults
api_wrapper
  field required (type=value_error.missing)

In [9]:
search.invoke("what is the weather in Hyderabad")

[{'url': 'https://weatherspark.com/h/m/109450/2024/1/Historical-Weather-in-January-2024-in-Hyderabad-India',
  'content': 'January 2024 Weather History in Hyderabad India  Observed Weather in January 2024 in Hyderabad Hours of Daylight and Twilight in January 2024 in Hyderabad  Hyderabad Temperature History January 2024 Hourly Temperature in January 2024 in Hyderabad  Compare Hyderabad to another city: Cloud Cover in January 2024 in Hyderabad72 °F comfortable Precipitation No Report Wind 4.6 mph light breeze Wind Dir. 120 deg, ESE Cloud Cover Mostly Clear 1,500 ft Raw: VOHY 170300Z 12004KT 1500 BR FEW015 22/15 Q1016 NOSIG This report shows the past weather for Hyderabad, providing a weather history for January 2024.'},
 {'url': 'https://www.weather2travel.com/india/hyderabad/january/',
  'content': 'Hyderabad weather in January 2024 Expect 29°C daytime maximum temperatures  weather averages for Hyderabad in January before you book your next holiday to India in 2024/2025.  Hyderabad Jan

In [11]:
url = LLMChain(llm=llm, prompt=prompt)

In [18]:
tools = [Tool.from_function(func = search.run, name="URL", description="Useful to find url for companies")]

In [37]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [42]:
agent.invoke({"input": "www.factset.com+leadership"})



> Entering new AgentExecutor chain...
I should use the search engine to find information about leadership on the website www.factset.com.
Action: tavily_search_results_json
Action Input: "leadership site:www.factset.com"
Observation: []
Thought:I couldn't find any results related to leadership on the website www.factset.com.
Thought: I should try a different search query.
Action: tavily_search_results_json
Action Input: "leadership factset"
Observation: [{'url': 'https://www.iese.edu/standout/management-trends-leadership-skills/', 'content': 'Management trends for 2024: Power skills for leaders  and fostering it is an essential power skill for 2024.  — that can make a difference in 2024.  4 essential power skills for managers in 2024Fostering connectedness and shared realities in the divisive and stressful world of 2024, especially when communication is largely online, is essential for companies. "Social bonds are the glue that hold organizations together. So if leaders can allow hum

{'input': 'www.factset.com+leadership',
 'output': 'The search results provide information about leadership trends in 2024, including power skills for leaders, the emergence of Gen X and Millennials in leadership roles, remote leadership skills, and the need for adaptability and collective thinking in leadership.'}

In [31]:
from langchain import hub
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
tavily_tool = TavilySearchResults()
tools = [tavily_tool]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [32]:
agent_executor.invoke({"input": "What happened in the latest burning man floods?"})



> Entering new AgentExecutor chain...


InvalidRequestError: Unrecognized request argument supplied: functions

In [35]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [36]:
prompt = "Write an essay in 1000 words for the topic {input}, use the tools to retrieve the necessary information"  
input = "Essay on Global Warming – Causes and Solutions"  
  
print(agent.run(prompt.format(input=input)))



> Entering new AgentExecutor chain...
Writing an essay requires gathering information on the topic. I can use the tavily_search_results_json tool to search for relevant information on the causes and solutions of global warming.
Action: tavily_search_results_json
Action Input: "Causes and solutions of global warming"
Observation: [{'url': 'https://www.washingtonpost.com/climate-environment/2024/01/02/record-heat-2024-el-nino/', 'content': 'What climate scientists are predicting for the globe in 2024  already assured. Now, some scientists are already speculating: 2024 could be even hotter.  More on climate change  world are becoming too hot to survive.4 min Comment As a year of surprising global warmth came to a close, a record high annual average temperature was already assured. Now, some scientists are already speculating: 2024 could be...'}, {'url': 'https://www.nature.com/articles/d41586-024-00074-z', 'content': "2023, as humans continue to pour heat-trapping greenhouse gases into 

In [44]:
template = "{input}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
chain.invoke({"input": "www.factset.com/+leadership"})

AIMessage(content="I'm sorry, but I am unable to browse the internet or access specific websites. However, if you have any questions or need information about leadership, I would be happy to help.")